<a href="https://colab.research.google.com/github/KamonohashiPerry/MachineLearning/blob/master/Causal_Inference/Python_Causal_Inference_Chap6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## LiNGAM(Linear Non-Gaussian Acyclic Model)
+ 構造方程式モデルを前提にしている。
 + 線形な構造方程式
 + 線形モデルの誤差項がガウス分布に従うノイズではない
 + DAGを扱うことを前提（循環しないということ）
  + 行列でいうところの下三角行列

## 独立成分分析
+ 主成分分析の発展形
+ 相関が0になったデータをさらに線形変換して、要素間の関係を独立にさせる操作。

### 擬似データの生成

In [3]:
# 乱数シードの固定
import random
import numpy as np
import pandas as pd

random.seed(1234)
np.random.seed(1234)

In [4]:
# データ数
num_data = 200

# 非ガウスのノイズ
ex1 = 2*(np.random.rand(num_data)-0.5)
ex2 = 2*(np.random.rand(num_data)-0.5)
ex3 = 2*(np.random.rand(num_data)-0.5)

# データ生成
x2 = ex2
x1 = 3*x2 + ex1
x3 = 2*x1 + 4*x2 + ex3

# 表にまとめる
df = pd.DataFrame({'x1':x1,
                     'x2':x2,
                     'x3':x3})
df.head()

,x1,x2,x3
0,2.257272,0.958078,8.776842
1,2.531611,0.762464,8.561263
2,0.641547,0.255364,1.341902
3,3.153636,0.860973,9.322791
4,1.908691,0.449580,5.776675


In [7]:
# 独立成分分析
from sklearn.decomposition import FastICA

ica = FastICA(random_state=1234, max_iter=10000).fit(df)

# ICAで求めた行列A
A_ica = ica.mixing_

# 行列Aの逆行列を求める
A_ica_inv = np.linalg.pinv(A_ica)

print(A_ica_inv)

[[-0.23203107 -0.4635971   0.1154553 ]
 [-0.02158245  0.12961253  0.00557934]
 [-0.11326384  0.40437635 -0.00563091]]


In [8]:
pip install munkres

In [9]:
from munkres import Munkres
from copy import deepcopy

In [11]:
# 絶対値の逆数にして対角成分の和を最小にする問題に置き換える
A_ica_inv_small = 1 / np.abs(A_ica_inv)

# 対角成分の和を最小にする行の入れ替え順を求める
m = Munkres() # ハンガリアン法
ixs = np.vstack(m.compute(deepcopy(A_ica_inv_small)))

# 求めた順番で変換
ixs = ixs[np.argsort(ixs[:, 0]), :]
ixs_perm = ixs[:, 1]
A_ica_inv_perm = np.zeros_like(A_ica_inv)
A_ica_inv_perm[ixs_perm] = A_ica_inv
print(A_ica_inv_perm)

# 並び替わった順番
print(ixs)

[[-0.11326384  0.40437635 -0.00563091]
 [-0.02158245  0.12961253  0.00557934]
 [-0.23203107 -0.4635971   0.1154553 ]]
[[0 2]
 [1 1]
 [2 0]]


In [12]:
# 行の大きさを調整
D = np.diag(A_ica_inv_perm)[:, np.newaxis]
A_ica_inv_perm_D = A_ica_inv_perm / D
print(A_ica_inv_perm_D)

[[ 1.         -3.57021564  0.04971498]
 [-0.16651518  1.          0.0430463 ]
 [-2.00970483 -4.01538182  1.        ]]


In [13]:
# B = I - A_inv
B_est = np.eye(3) - A_ica_inv_perm_D
print(B_est)

[[ 0.          3.57021564 -0.04971498]
 [ 0.16651518  0.         -0.0430463 ]
 [ 2.00970483  4.01538182  0.        ]]


In [15]:
# scikit-learnから線形回帰をimport
from sklearn.linear_model import LinearRegression

# 説明変数
X1 = df[['x2']]
X3 = df[['x1', 'x2']]

# 回帰の実施
reg1 = LinearRegression().fit(X1, df['x1'])
reg3 = LinearRegression().fit(X3, df['x3'])

# 回帰した結果の係数を出力
print('係数：', reg1.coef_)
print('係数：', reg3.coef_)

係数： [3.14642595]
係数： [1.96164568 4.11256441]
